<a href="https://colab.research.google.com/github/dominikklepl/News-search-engine/blob/master/Code_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News search engine
In this notebook we'll go through all code for building our own search engine specialised on news articles.
Each search engine consists of three main components:


* Crawler
* Indexer
* Query processor



## A. Crawler
We start with building crawler that goes to RSS feed, extracts title, description (if any), date published and link. Processes this information and saves in a meta-data "database" (actually a pandas dataframe and stores it as csv file.

### Import packages

In [11]:
!pip install feedparser

import feedparser
import pandas as pd

Read RSS feed from a URL. Also keep track of how many news are in the feed and how many of these have already been scraped with previous iteration of the crawler. This will be later important to automatically adjust the frequency with which the crawler will visit the RSS feed to scrape new information.
For testing we use just one URL: BBC World News.

In [12]:
URL = "http://feeds.bbci.co.uk/news/world/rss.xml"
feed = feedparser.parse(URL)

feed_len = len(feed.entries) #number of news in feed
old_news = 0  # count how many news in feed were already scraped

print("There are {} news in the RSS feed." .format(feed_len))

There are 32 news in the RSS feed.


Load the meta-data database stored as csv file. If this is the first time the crawler is let loose this will be just an empty file with prepared column names (ID, title, date and link).
I'll keep it commented out here and instead just create an empty dataframe a this point.

In [13]:
#meta_data = pd.read_csv(PATH + "database.csv", index_col = 'Unnamed: 0')
meta_data = pd.DataFrame(columns=['ID', 'title', 'summary', 'link', 'published'])
meta_data.head()

,ID,title,summary,link,published


Now we write a function that accepts one entry from the feed and parse its contents to list with title, date published and link and assigns it a unique ID (which also denotes when was the entry scraped and entered to our search engine).
We normalize the date so that it's in format day/month/year. We don't care about more precise time.

In [0]:
def process_entry(entry, ID):
  ID = ID
  title = entry.title
  summary = entry.summary
  link = entry.link
  published = str(entry.published_parsed.tm_mday) + '/' + \
              str(entry.published_parsed.tm_mon) + '/' + \
              str(entry.published_parsed.tm_year)
  return [ID, title, summary, link, published]

Test the function on one entry

In [15]:
test_entry = feed.entries[0]
process_entry(test_entry, 1)

[1,
 'Iran petrol price hike: Protests erupt over surprise rationing',
 'Several cities see protesters take to the streets as petrol prices go up by at least 50%.',
 'https://www.bbc.co.uk/news/world-middle-east-50444429',
 '16/11/2019']

Now we iterate over all entries in the feed. Check if the entry is already in the meta-data database, if not the entry is processed, assigned an ID and appended to a list of entries that will be later added to the database.

In [0]:
data = [] #dataframe for saving the entries
n = len(meta_data)+1 #ID value based on the highest ID value in database
for i in range(len(feed.entries)):
  entry = feed.entries[i]
  
  #check that link isn't in the database yet
  if entry.link not in meta_data['link'].values:
    processed = process_entry(entry = entry, ID=n)
    data.append(processed)
    n += 1 #increase the ID value
  else: old_news += 1 #count already scraped entries

If there was at least one newly scraped entry, we add it to the database.

In [0]:
if len(data) > 0:
  #transform data to pandas DataFrame
  news_extracted = pd.DataFrame(data, columns=['ID', 'title', 'summary', 'link', 'published'])

  #add new news to the database
  meta_data = pd.concat([meta_data, news_extracted], axis = 0)

  #write database to a csv file
  #meta_data.to_csv(PATH + "database.csv")

Look at the database

In [18]:
meta_data.head(5)

,ID,title,summary,link,published
0,1,Iran petrol price hike: Protests erupt over su...,Several cities see protesters take to the stre...,https://www.bbc.co.uk/news/world-middle-east-5...,16/11/2019
1,2,Impeachment inquiry: Trump 'asked for probe in...,A US diplomatic aide says he overheard an envo...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019
2,3,Hong Kong protests: Chinese soldiers clean up ...,Chinese soldiers in Hong Kong have left their ...,https://www.bbc.co.uk/news/world-asia-china-50...,16/11/2019
3,4,Sri Lanka votes for president in shadow of Eas...,Terror fears could see the defence chief who o...,https://www.bbc.co.uk/news/world-asia-50013478,16/11/2019
4,5,Rodney Reed: Texas court halts execution in hi...,"Rodney Reed had been set to die in days, but l...",https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019


Get percentage of already-scraped entries

In [19]:
print("{} % of entries were already scraped." .format((old_news/feed_len)*100))

0.0 % of entries were already scraped.


## B. Indexer
Second part of a search engine is an indexer. It's basically a smart storage of our news articles in which we can later easily retrieve relative articles given a search query.
It parses the title and description of the news articles scraped by the crawler to single words. All these words make up the vocabulary of our index. Next step is to put the ID of the article in the posting lists of the words that the article contains. For example article called "This happened today" will be stored in posting lists of terms "this", "happened" and "today".
Before creating the index we preprocess the text of the articles in order to get rid of useless information. We the text of accents and turn everything to lowercase. Next we perform lemmatization. This is slightly smarter version of stemming. Essentially, it's a word normalization, e.g. all nouns to singular, all verbs in present tense etc.

Let's do it.

#### Importing

In [20]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


We start with an empty dictionary as our index. As we scrape more articles later we will instead of starting with an empty index just update the already created index.

The index is organized as:
```
{
  "word1": \[ID1, ID2, ...],
  "word2": \[ID5, ID8, ...],
  ...
}
```

We start with just a single article from our meta-data database.

In [0]:
entry = meta_data.loc[0,:].copy()

In [22]:
entry

ID                                                           1
title        Iran petrol price hike: Protests erupt over su...
summary      Several cities see protesters take to the stre...
link         https://www.bbc.co.uk/news/world-middle-east-5...
published                                           16/11/2019
Name: 0, dtype: object

### Text preprocessing
Turn title to lowercase, remove accents

In [0]:
def process_string(text):
  text = text.lower() #to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  return text

In [24]:
process_string(entry.title)

'iran petrol price hike protests erupt over surprise rationing'

Now, lemmatize, i.e. word normalization.

This method requires some additional information about the words. We need to find the word category of each word, e.g. verb, noun etc.

In [0]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

Test the function

In [26]:
print("Apple: {}\n Run: {}\n Happy: {}" .format(get_wordnet_pos("apple"), get_wordnet_pos("run"), get_wordnet_pos("happy")))

Apple: n
 Run: v
 Happy: a


We also need to remove stopwords, i.e. words with low informational value.

In [0]:
stop = stopwords.words('english')

Now we'll iterate over all words in text, lemmatize and return the transformed string.

In [0]:
lem = WordNetLemmatizer()

def stop_lemmatize(doc):
    tokens = nltk.word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in stop:
            tmp += lem.lemmatize(w, get_wordnet_pos(w)) + " "
    return tmp

In [29]:
stop_lemmatize(doc = entry.title)

'Iran petrol price hike : Protests erupt surprise ration '

In [0]:
def process_string(text):
  text = text.lower() #to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  text = stop_lemmatize(text)
  return text

In [31]:
%time process_string(entry.title)

CPU times: user 3.42 ms, sys: 70 µs, total: 3.49 ms
Wall time: 3.9 ms


'iran petrol price hike protest erupt surprise ration '

Now we apply the process_string function to all titles and summaries in our database.

In [0]:
meta_processed = meta_data.copy()

In [0]:
def transform_df(df):
  df['title'] = df['title'].apply(process_string)
  df['summary'] = df['summary'].apply(process_string)

In [34]:
%time transform_df(meta_processed)

CPU times: user 140 ms, sys: 4.74 ms, total: 144 ms
Wall time: 148 ms


In [35]:
meta_processed.head(5)

,ID,title,summary,link,published
0,1,iran petrol price hike protest erupt surprise ...,several city see protester take street petrol ...,https://www.bbc.co.uk/news/world-middle-east-5...,16/11/2019
1,2,impeachment inquiry trump ask probe ukraine en...,u diplomatic aide say overheard envoy phone ca...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019
2,3,hong kong protest chinese soldier clean street,chinese soldier hong kong left barrack help di...,https://www.bbc.co.uk/news/world-asia-china-50...,16/11/2019
3,4,sri lanka vote president shadow easter sunday ...,terror fear could see defence chief oversaw de...,https://www.bbc.co.uk/news/world-asia-50013478,16/11/2019
4,5,rodney reed texas court halt execution highpro...,rodney reed set die day lawyer say new evidenc...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019


In practice, we won't be transforming the whole meta-data database since that would mean creating index from scratch after every crawler iteration. Instead we would use only subset of the database with only newly added articles.

Now we can iterate over all entries to create the index. We'll go step by step again before wrapping it all in one nice function.

Merge title and summary into one field and drop all columns except for ID as we don't need those anymore.

In [0]:
meta_processed['text'] = meta_processed['title'] + " " + meta_processed['summary']
drop_cols = ['title', 'summary', 'published', 'link']
meta_processed = meta_processed.drop(drop_cols, axis=1)

In [37]:
meta_processed.head(5)

,ID,text
0,1,iran petrol price hike protest erupt surprise ...
1,2,impeachment inquiry trump ask probe ukraine en...
2,3,hong kong protest chinese soldier clean street...
3,4,sri lanka vote president shadow easter sunday ...
4,5,rodney reed texas court halt execution highpro...


Add this part to a transform_df function.

In [0]:
def transform_df(df):
  df = df
  df['title'] = df['title'].apply(process_string)
  df['summary'] = df['summary'].apply(process_string)
  df['text'] = df['title'] + " " + df['summary']
  drop_cols = ['title', 'summary', 'published', 'link']
  df = df.drop(drop_cols, axis=1)
  return df

### Build index

Now we'll build index with just one entry.

In [39]:
entry = meta_processed.loc[0,:].copy()
print(entry)
index_test = {}

ID                                                      1
text    iran petrol price hike protest erupt surprise ...
Name: 0, dtype: object


Split the entry to single words and return list and save entry's ID as object.

In [0]:
words = entry.text.split()
ID = entry.ID

Each word in index' vocabulary is a dictionary key and has its own posting list with IDs. Let's construct one word vocabulary as example.

In [41]:
word = words[0]
sample = {word: [ID]}
print(sample)

{'iran': [1]}


Now we iterate over all words and if they aren't in the vocabulary yet we add them. Also for each word we append the entry ID to the posting list.

In [0]:
for word in words:
  if word in index_test.keys():
    if ID not in index_test[word]:
      index_test[word].append(ID)
  else:
    index_test[word] = [ID]

In [43]:
print(index_test)

{'iran': [1], 'petrol': [1], 'price': [1], 'hike': [1], 'protest': [1], 'erupt': [1], 'surprise': [1], 'ration': [1], 'several': [1], 'city': [1], 'see': [1], 'protester': [1], 'take': [1], 'street': [1], 'go': [1], 'least': [1], '50': [1]}


Now this process can be repeated for all entries in the database

In [0]:
def index_it(entry, index):
  words = entry.text.split()
  ID = entry.ID
  for word in words:
    if word in index.keys():
      if ID not in index[word]:
        index[word].append(ID)
    else:
      index[word] = [ID]
  return index

In [45]:
ind = index_it(entry=entry, index= {})
print(ind)

{'iran': [1], 'petrol': [1], 'price': [1], 'hike': [1], 'protest': [1], 'erupt': [1], 'surprise': [1], 'ration': [1], 'several': [1], 'city': [1], 'see': [1], 'protester': [1], 'take': [1], 'street': [1], 'go': [1], 'least': [1], '50': [1]}


Again we can iterate over all entries in the database with scraped articles, process them append to index.

In [0]:
def index_all(df, index):
  for i in range(len(df)):
    entry = df.loc[i,:]
    index = index_it(entry = entry, index = index)
  return index

In [47]:
index = index_all(meta_processed, index = {})
len(index)

364

Finally we wrap everything in one nice function.

In [0]:
def build_index(df, index):
    to_add = transform_df(df)
    index = index_all(df = to_add, index = index)
    return index

In [0]:
idx = build_index(df = meta_data, index = {})

In [50]:
len(idx)

364

And for future use we save the index to json file.

In [0]:
import json

with open('index.json', 'w') as fp:
    json.dump(idx, fp, sort_keys=True, indent=4)

It can be of course opened again with following code.

In [0]:
with open('index.json', 'r') as f:
    data = json.load(f)

# Bb Ranked retrieval
  The user would probably prefer the more relevant pages to be displayed before those that are less relevant (hopefully they're at least a bit relevant).
For our search engine to support such option we need to store some information about the scraped documents that could be later used for this purpose.
We'll use averaged word2vec for this purpose. Word2Vec model is single hidden-layer neural network. The hidden layer is actually what is so useful about this model. Given a word the layer's activation gives a unique vector that word. For each document we can iterate over all words, extract their vectors and then by averaging obtain a document vector. 
  Compared to other methods averaged word2vec has multiple advantages. Unlike simpler methods such as bag-of-words, n-grams and tf-idf the size of the vectors is fixed. For example bag-of-words is also using vectors but the size of these vectors equals the number of unique words in the corpus. This means that the computational and storage requirements get larger as the corpus gets larger.
  Averaged word2vec is also able to represent the documents on more abstract level than simpler methods and should therefore provide better method of ranking.
  We're using word2vec rather than doc2vec because we can simply use pretrained word2vec model to compute the document vectors. Using doc2vec would mean training a neural network from scratch which requires computational power, time and rather large dataset. 

Import and download pretrained word2vec model

In [69]:
import gensim
import numpy as np
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2019-11-16 16:06:28--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.98.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.98.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  45.3MB/s    in 30s     

2019-11-16 16:06:58 (52.9 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



Load word2vec model

In [70]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Try getting vectors for all words in the text and averaging to get single vector.

In [71]:
print(words)

['iran', 'petrol', 'price', 'hike', 'protest', 'erupt', 'surprise', 'ration', 'several', 'city', 'see', 'protester', 'take', 'street', 'petrol', 'price', 'go', 'least', '50']


In [0]:
def average_vectors(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    if len(doc) == 0:
      return np.zeros(300)
    else:
      return np.mean(word2vec_model[doc], axis=0)

In [121]:
%time test_vec = average_vectors(word2vec, words)

CPU times: user 1.25 ms, sys: 13 µs, total: 1.26 ms
Wall time: 1.44 ms


Now we can iterate over documents, compute their vectors and construct a document vectors database.

In [0]:
def prepare_ranking(df):
  corpus = df[['ID', 'text']].copy()
  doc_vecs = {}
  for i in range(len(corpus)):
    row = corpus.loc[i,:]
    text = row.text.split()
    doc_vecs[row.ID]=average_vectors(word2vec, text)
  return doc_vecs

In [103]:
doc_vecs = prepare_ranking(df=meta_data)

CPU times: user 20.4 ms, sys: 3.98 ms, total: 24.4 ms
Wall time: 32.1 ms


## C. Query processor
The final part of a search engine is a query processor which actually performs the search task. Given a query by user the processor should return list of relevant documents.
There are multiple types of queries. We'll start with a simple "google-ish" query where assume the user looks for documents relevant to all words in the query. Therefore we transform the query to boolean by connecting all words with AND operator.

First, the processor preprocesses the query the same way as the indexer preprocessed the text. In other words, we normalize the query to match the format of text in the index. Next, the query is parsed to single words. We look into index if these words are part of the vocabulary. If a word is in index we retrieve its posting list. Finally, we look for intersection of all retrieved posting lists. The result is list of document IDs that the user asked for.
However, we need to return something more useful than just a list of IDs. Therefore,we retrieve the information stored about the documents in the meta-data database. Before printing the results we should also rank the documents. This ranking should be based on relevance to query.
Optionally, the user may ask for news only from limited time window, e.g. published today or last week. So we need to filter the retrieved documents if this happens.

-----
To implement:
 - Boolean query
 - phrase matching
 -----


### Normalize query

Let's define an example query.

In [0]:
test = "Trump Ukraine China"

Now we use the "process string" function from used by indexer to normalize the query

In [55]:
print("User query: {}." .format(test))
test_norm = process_string(test)
print("Normalized query: {}." .format(test_norm))

User query: Trump Ukraine China.
Normalized query: trump ukraine china .


Now we split the query into words.

In [0]:
test_split = test_norm.split()

And we wrap this in function

In [0]:
def process_query(query):
  norm = process_string(query)
  return norm.split()

### Retrieve from index

And we iterate over the words, looking if they're in the index vocabulary. If so then we retrieve the associated posting list.

In [0]:
retrieved = []
for word in test_split:
  if word in index.keys():
    retrieved.append(index[word])

Now we look for the intersection of all posting lists.

In [0]:
def lists_intersection(lists):
  intersect = list(set.intersection(*map(set, lists)))
  intersect.sort()
  return intersect

In [60]:
result = lists_intersection(retrieved)
print(result)

[2, 15, 16, 23]


Let's wrap this part in a function before proceeding to formatting the results.

In [0]:
def retrieve_common(query_split, index):
  retrieved = []
  for word in query_split:
    if word in index.keys():
      retrieved.append(index[word])
  result = lists_intersection(retrieved)
  return result

In [62]:
result_IDs = retrieve_common(test_split, index)
print(result_IDs)

[2, 15, 16, 23]


-----

*TO DO:
If there's no document retrieved, try removing one term and looking for simplified query + tell user that such document doesn't include term X.*

-----

### Retrieve meta-data
Now we need to connect the retrieved IDs with some useful information stored in database that we first use to refine the results and then to print nice result to user.

In [63]:
#in real setting we'll read the database from file here
#meta = pd.read_csv("database.csv")

#this is our database
meta = meta_data.drop(['text'], axis=1).copy()
meta.head(5)

,ID,title,summary,link,published
0,1,iran petrol price hike protest erupt surprise ...,several city see protester take street petrol ...,https://www.bbc.co.uk/news/world-middle-east-5...,16/11/2019
1,2,impeachment inquiry trump ask probe ukraine en...,u diplomatic aide say overheard envoy phone ca...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019
2,3,hong kong protest chinese soldier clean street,chinese soldier hong kong left barrack help di...,https://www.bbc.co.uk/news/world-asia-china-50...,16/11/2019
3,4,sri lanka vote president shadow easter sunday ...,terror fear could see defence chief oversaw de...,https://www.bbc.co.uk/news/world-asia-50013478,16/11/2019
4,5,rodney reed texas court halt execution highpro...,rodney reed set die day lawyer say new evidenc...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019


Query from database to get only rows of retrieved IDs

In [64]:
result_meta = meta[meta.ID.isin(result_IDs)].reset_index(drop=True)
result_meta.head(5)

,ID,title,summary,link,published
0,2,impeachment inquiry trump ask probe ukraine en...,u diplomatic aide say overheard envoy phone ca...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019
1,15,trump impeachment inquiry ukraine important u,relation eastern european country become centr...,https://www.bbc.co.uk/news/world-europe-50419668,15/11/2019
2,16,trump impeachment who ukraine story,story kickstarted impeachment move donald trum...,https://www.bbc.co.uk/news/world-us-canada-498...,7/11/2019
3,23,trump livetweets attack yovanovitch interrupt ...,former ukraine ambassador responds president t...,https://www.bbc.co.uk/news/world-us-canada-504...,15/11/2019


### Date filtering
User might ask for news from specific day or date range.
For simplicity let's assume the user enters date in format day/month/year. 
Now we can define 2 types of date restrictions:
* single day
* date range - from X/X/X - to X/X/X

Another option of date restriction could be saying 'today'.



Restricing to single day

In [65]:
test = "16/11/2019"

#get news published on "test"
results_single = result_meta[result_meta.published==test].reset_index(drop=True)
results_single.head()

,ID,title,summary,link,published
0,2,impeachment inquiry trump ask probe ukraine en...,u diplomatic aide say overheard envoy phone ca...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019


Restricing to "today"

In [66]:
#get today's date
from datetime import date, timedelta

def get_today():
  today = date.today()
  today = today.strftime("%d/%m/%Y")
  return today

results_today = result_meta[result_meta.published==get_today()].reset_index(drop=True)
results_today.head()

,ID,title,summary,link,published
0,2,impeachment inquiry trump ask probe ukraine en...,u diplomatic aide say overheard envoy phone ca...,https://www.bbc.co.uk/news/world-us-canada-504...,16/11/2019


Restricting to time interval

In [0]:
def daterange(start, end):
    for n in range(int ((end - start).days)+1):
        yield start + timedelta(n)

def format_date(dt):
  dt = dt.split("/")
  dt = date(int(dt[2]), int(dt[1]), int(dt[0]))
  return(dt)

def date_interval(interval):
  interval = interval.split("-")
  start = format_date(interval[0])
  end = format_date(interval[1])
  interval = []
  for dt in daterange(start, end):
      interval.append(dt.strftime("%d/%m/%Y"))
  return interval

In [68]:
date_interval("15/11/2019 - 16/11/2019")

['15/11/2019', '16/11/2019']